In [1]:
import requests               #to request list of documents in htlm format
from bs4 import BeautifulSoup #to parse html
from datetime import datetime #to convert date to ISO8601
import pandas as pd           #to save it as csv. 
import urllib.request         #to download documents
import time                   #to give doc downloaded time to operate

# TODO:
# Change use of pandas to save to a csv file (use csv module instead)

In [2]:
DEBUG_FLAG = False

print("Loading website:")
if DEBUG_FLAG:
    r = open('./test-files/Documents _ The Financial Oversight and Management Board for Puerto Rico.html')
    soup = BeautifulSoup(r, 'html.parser')
else:
    tries_count = 0
    while True:
        try:
            r = requests.get('https://juntasupervision.pr.gov/index.php/en/documents/')
            break
        except:
            tries_count += 1
            if tries_count>10:
                print("Already tried 10 times...I'm giving up. Sorry.")
                break
            else:
                print("Could not access the website. Trying again...")
    soup = BeautifulSoup(r.content, 'html.parser')
print("Done.")

Loading website:
Could not access the website. Trying again...
Could not access the website. Trying again...
Done.


In [3]:
# Documents are placed in each row, so go straight to it

doc_list_soup = soup.findAll("div",{"class": "doc-row"});

In [4]:
# Store every document in a table row

table = []

for doc in doc_list_soup:
    title = doc.a.getText()
    
    category = doc.find("div",{"class": "span2 cat"}).getText()
    
    tmp_date = doc.find("div",{"class": "span2 date"}).getText()
    date = str(datetime.strptime(tmp_date, '%b.%d.%Y')) # convert to ISO-8601 date
    
    download_url = doc.a.get('href')
    
    download_title = doc.a.get('download')
    
    table.append( [title,category,date,download_url,download_title] )

In [5]:
# Store data to a csv file

csv_file = './new-downloaded-docs/doc-table.csv'

df = pd.DataFrame(table)
df.columns = ['title','category','date','download_url','download_title']
df.to_csv(csv_file,index=False)

In [ ]:
# Download documents

path = "./new-downloaded-docs/"

num_files = len(df.download_url)

for url,i in zip(df.download_url, range(num_files)):
    fn = url.split('/')[-1]    
    tries_count = 0
    
    while True:
        try:
            print("Downloading file #" + str(i+1) + "/" + str(num_files) + " - " + fn)
            urllib.request.urlretrieve(url, filename=path+fn)
            break
        except:
            tries_count += 1
            if tries_count>10:
                print("Already tried 10 times...I'm giving up. Sorry")
            else:
                print("Could not download file. Trying again.")

Could not download file. Trying again.
Could not download file. Trying again.
Could not download file. Trying again.
Could not download file. Trying again.
Could not download file. Trying again.
Could not download file. Trying again.
Could not download file. Trying again.
Could not download file. Trying again.
Could not download file. Trying again.
Could not download file. Trying again.
Could not download file. Trying again.
Could not download file. Trying again.
Could not download file. Trying again.
Could not download file. Trying again.
Could not download file. Trying again.
Could not download file. Trying again.
Could not download file. Trying again.
Could not download file. Trying again.
Could not download file. Trying again.
Could not download file. Trying again.
Could not download file. Trying again.
Could not download file. Trying again.
Could not download file. Trying again.
Could not download file. Trying again.
Could not download file. Trying again.
Could not download file. 